# Deep Learning

In this notebook, I'll work with the churn dataset and use ANN and CNN to detect patterns.

I'll first get the dataset for CNN, extract it and then setup the GPU usage.

In [0]:
!wget https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P14-Convolutional-Neural-Networks.zip

In [0]:
!unzip P14-Convolutional-Neural-Networks.zip

In [0]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

## Artificial Neural Networks

### Data Processing

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1]
y = dataset.iloc[:, -1]

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelEncoder_1 = LabelEncoder()
X.iloc[:, 1] = labelEncoder_1.fit_transform(X.iloc[:, 1])
labelEncoder_2 = LabelEncoder()
X.iloc[:, 2] = labelEncoder_2.fit_transform(X.iloc[:, 2])
oneHotEncoder = OneHotEncoder(categorical_features = [1])
X = oneHotEncoder.fit_transform(X).toarray()

# Avoid dummy variable trap
X = X[:, 1:]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [0]:
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()
X_train = standardScaler.fit_transform(X_train)
X_test = standardScaler.transform(X_test)

### Build the ANN

In [9]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
import keras.backend as K
K.set_image_dim_ordering('tf')
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [10]:
ann_classifier = Sequential()

# Nodes count = average of input and output values

# First hidden layer
ann_classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = X_train.shape[1])) # relu is for rectifier
# Second hidden layer
ann_classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu')) # relu is for rectifier
# Output layer
ann_classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

ann_classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # adam is an optimized scholastic gradient descent

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  # Remove the CWD from sys.path while we load stuff.


In [11]:
ann_classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
8000/8000 [==============================] - 4s 470us/step - loss: 0.4846 - acc: 0.7955
Epoch 2/100
8000/8000 [==============================] - 3s 416us/step - loss: 0.4271 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 3s 416us/step - loss: 0.4217 - acc: 0.7975
Epoch 4/100
8000/8000 [==============================] - 3s 415us/step - loss: 0.4186 - acc: 0.8224
Epoch 5/100
8000/8000 [==============================] - 3s 417us/step - loss: 0.4164 - acc: 0.8262
Epoch 6/100
8000/8000 [==============================] - 3s 417us/step - loss: 0.4145 - acc: 0.8289
Epoch 7/100
8000/8000 [==============================] - 3s 430us/step - loss: 0.4130 - acc: 0.8329
Epoch 8/100
8000/8000 [==============================] - 4s 440us/step - loss: 0.4118 - acc: 0.8314
Epoch 9/100
8000/8000 [==============================] - 4s 439us/step - loss: 0.4110 - acc: 0.8332
Epoch 10/100
8000/8000 [============================

### Making predictions

In [0]:
y_pred = ann_classifier.predict(X_test)
y_pred = y_pred > 0.5

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score

print("Accuracy: {}%".format(accuracy_score(y_test, y_pred)*100))
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix: ")
print("{}".format(cm))

Accuracy: 83.95%
Confusion matrix: 
[[1538   57]
 [ 264  141]]


## Convolutional Neural Network

In [0]:
from keras.layers import Convolution2D, MaxPooling2D, Flatten

### Build CNN

1. Convolution
2. Max Pooling
3. Flatten
4. Full Connection

In [0]:
classifier = Sequential()
classifier.add(Convolution2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2), data_format="channels_last"))
classifier.add(Convolution2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2), data_format="channels_last"))
classifier.add(Flatten())
classifier.add(Dense(activation = 'relu', units=128))
classifier.add(Dense(activation = 'sigmoid', units=1))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Image Augmentation

In [17]:
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
                 rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
train_generator = ImageDataGenerator(rescale = 1./255)

training_set = train_generator.flow_from_directory('Convolutional_Neural_Networks/dataset/training_set', 
                                                    target_size = (64, 64), 
                                                    batch_size = 32, 
                                                    class_mode = 'binary')

test_set = train_generator.flow_from_directory('Convolutional_Neural_Networks/dataset/test_set', 
                                                target_size = (64, 64), 
                                                batch_size = 32, 
                                                class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


### Training the model

In [19]:
classifier.fit_generator(training_set, 
                         steps_per_epoch=250, 
                         validation_data=test_set, 
                         use_multiprocessing=True, 
                         verbose=1,
                         epochs=25, 
                         validation_steps=2000)

Epoch 1/25
250/250 [==============================] - 216s 866ms/step - loss: 0.6319 - acc: 0.6454 - val_loss: 0.5811 - val_acc: 0.6955
Epoch 2/25
250/250 [==============================] - 216s 865ms/step - loss: 0.5309 - acc: 0.7329 - val_loss: 0.5328 - val_acc: 0.7375
Epoch 3/25
250/250 [==============================] - 216s 862ms/step - loss: 0.4710 - acc: 0.7756 - val_loss: 0.5130 - val_acc: 0.7564
Epoch 4/25
250/250 [==============================] - 216s 862ms/step - loss: 0.4268 - acc: 0.8035 - val_loss: 0.5208 - val_acc: 0.7620
Epoch 5/25
250/250 [==============================] - 216s 864ms/step - loss: 0.3714 - acc: 0.8327 - val_loss: 0.5028 - val_acc: 0.7766
Epoch 6/25
250/250 [==============================] - 215s 862ms/step - loss: 0.3072 - acc: 0.8652 - val_loss: 0.5506 - val_acc: 0.7559
Epoch 7/25
250/250 [==============================] - 216s 863ms/step - loss: 0.2464 - acc: 0.8949 - val_loss: 0.6065 - val_acc: 0.7673
Epoch 8/25
250/250 [============================

Using **Convolutional Neural Networks**, I was able to train a model to detect between cats and dogs.